# Defaults

## 비디오 뼈대

### 라이브러리

In [2]:
import cv2
import numpy as np
import face_recognition
import os
import sys
import mediapipe as mp
import c3d # C3D library import
import open3d as o3d # Open3D 임포트
import time # time 모듈 임포트
from filterpy.kalman import KalmanFilter

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### 전역 변수

In [29]:
# --- Global MediaPipe and Landmark Mappings ---
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
model_complexity = 2
min_detection_confidence = 0.9
min_tracking_confidence = 0.9
min_draw_visibility = 0.2

apply_kalman_filter = True
kalman_filter_process_noise_std=0.001
kalman_filter_measurement_noise_std = 0.05
output_dir = 'result_data'

#### 그림용


In [5]:
# MediaPipe Landmark Names for C3D Marker Mapping
MEDIAPIPE_LANDMARK_NAMES = {
    mp_pose.PoseLandmark.NOSE.value: 'NOSE',
    mp_pose.PoseLandmark.LEFT_EYE_INNER.value: 'L_EYE_INNER',
    mp_pose.PoseLandmark.LEFT_EYE.value: 'L_EYE',
    mp_pose.PoseLandmark.LEFT_EYE_OUTER.value: 'L_EYE_OUTER',
    mp_pose.PoseLandmark.RIGHT_EYE_INNER.value: 'R_EYE_INNER',
    mp_pose.PoseLandmark.RIGHT_EYE.value: 'R_EYE',
    mp_pose.PoseLandmark.RIGHT_EYE_OUTER.value: 'R_EYE_OUTER',
    mp_pose.PoseLandmark.LEFT_EAR.value: 'L_EAR',
    mp_pose.PoseLandmark.RIGHT_EAR.value: 'R_EAR',
    mp_pose.PoseLandmark.MOUTH_LEFT.value: 'L_MOUTH',
    mp_pose.PoseLandmark.MOUTH_RIGHT.value: 'R_MOUTH',
    mp_pose.PoseLandmark.LEFT_SHOULDER.value: 'L_SHOULDER',
    mp_pose.PoseLandmark.RIGHT_SHOULDER.value: 'R_SHOULDER',
    mp_pose.PoseLandmark.LEFT_ELBOW.value: 'L_ELBOW',
    mp_pose.PoseLandmark.RIGHT_ELBOW.value: 'R_ELBOW',
    mp_pose.PoseLandmark.LEFT_WRIST.value: 'L_WRIST',
    mp_pose.PoseLandmark.RIGHT_WRIST.value: 'R_WRIST',
    mp_pose.PoseLandmark.LEFT_PINKY.value: 'L_PINKY',
    mp_pose.PoseLandmark.RIGHT_PINKY.value: 'R_PINKY',
    mp_pose.PoseLandmark.LEFT_INDEX.value: 'L_INDEX',
    mp_pose.PoseLandmark.RIGHT_INDEX.value: 'R_INDEX',
    mp_pose.PoseLandmark.LEFT_THUMB.value: 'L_THUMB',
    mp_pose.PoseLandmark.RIGHT_THUMB.value: 'R_THUMB',
    mp_pose.PoseLandmark.LEFT_HIP.value: 'L_HIP',
    mp_pose.PoseLandmark.RIGHT_HIP.value: 'R_HIP',
    mp_pose.PoseLandmark.LEFT_KNEE.value: 'L_KNEE',
    mp_pose.PoseLandmark.RIGHT_KNEE.value: 'R_KNEE',
    mp_pose.PoseLandmark.LEFT_ANKLE.value: 'L_ANKLE',
    mp_pose.PoseLandmark.RIGHT_ANKLE.value: 'R_ANKLE',
    mp_pose.PoseLandmark.LEFT_HEEL.value: 'L_HEEL',
    mp_pose.PoseLandmark.RIGHT_HEEL.value: 'R_HEEL',
    mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value: 'L_FOOT_INDEX',
    mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value: 'R_FOOT_INDEX',
}

ALL_POSE_LANDMARK_INDICES = list(range(len(mp_pose.PoseLandmark)))
ALL_POSE_MARKER_NAMES = [MEDIAPIPE_LANDMARK_NAMES.get(i, f'UNKNOWN_{i}') for i in ALL_POSE_LANDMARK_INDICES]

# --- Color Definitions for Drawing ---
COLOR_LEFT_ARM = (255, 0, 0)     # Blue (Left Arm)
COLOR_RIGHT_ARM = (0, 0, 255)    # Red (Right Arm)
COLOR_LEFT_LEG = (255, 255, 0)   # Cyan (Left Leg)
COLOR_RIGHT_LEG = (0, 255, 255)  # Yellow (Right Leg)
COLOR_TORSO = (0, 255, 0)        # Green (Torso)
COLOR_HEAD_NECK = (255, 255, 255) # White (Head/Neck)

# Map connections to their respective colors
CONNECTIONS_COLORS = {
    # Arms
    (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_ELBOW.value): COLOR_LEFT_ARM,
    (mp_pose.PoseLandmark.LEFT_ELBOW.value, mp_pose.PoseLandmark.LEFT_WRIST.value): COLOR_LEFT_ARM,
    (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_ELBOW.value): COLOR_RIGHT_ARM,
    (mp_pose.PoseLandmark.RIGHT_ELBOW.value, mp_pose.PoseLandmark.RIGHT_WRIST.value): COLOR_RIGHT_ARM,
    
    # Legs
    (mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value): COLOR_LEFT_LEG,
    (mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value): COLOR_LEFT_LEG,
    (mp_pose.PoseLandmark.LEFT_ANKLE.value, mp_pose.PoseLandmark.LEFT_HEEL.value): COLOR_LEFT_LEG,
    (mp_pose.PoseLandmark.LEFT_HEEL.value, mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value): COLOR_LEFT_LEG,
    (mp_pose.PoseLandmark.RIGHT_HIP.value, mp_pose.PoseLandmark.RIGHT_KNEE.value): COLOR_RIGHT_LEG,
    (mp_pose.PoseLandmark.RIGHT_KNEE.value, mp_pose.PoseLandmark.RIGHT_ANKLE.value): COLOR_RIGHT_LEG,
    (mp_pose.PoseLandmark.RIGHT_ANKLE.value, mp_pose.PoseLandmark.RIGHT_HEEL.value): COLOR_RIGHT_LEG,
    (mp_pose.PoseLandmark.RIGHT_HEEL.value, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value): COLOR_RIGHT_LEG,

    # Torso
    (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_SHOULDER.value): COLOR_TORSO,
    (mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.RIGHT_HIP.value): COLOR_TORSO,
    (mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value): COLOR_TORSO,
    (mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_HIP.value): COLOR_TORSO,
    
    # Head/Neck (Nose to shoulders to represent neck for simplified face)
    (mp_pose.PoseLandmark.NOSE.value, mp_pose.PoseLandmark.LEFT_SHOULDER.value): COLOR_HEAD_NECK, 
    (mp_pose.PoseLandmark.NOSE.value, mp_pose.PoseLandmark.RIGHT_SHOULDER.value): COLOR_HEAD_NECK,
}

### Helper Functions

In [6]:
# --- 3D 랜드마크 시각화 함수 (Open3D 사용) ---
def visualize_3d_landmarks_animation(all_frames_3d_landmarks, connections, fps):
    """
    수집된 3D 랜드마크 데이터를 Open3D를 사용하여 애니메이션으로 시각화합니다.

    Args:
        all_frames_3d_landmarks (list): 각 프레임의 3D 랜드마크 (Numpy 배열 리스트).
                                        각 배열은 (num_markers, 3) 형태. (미터 단위)
        connections (list): 랜드마크 연결 정보를 담은 튜플 리스트 (예: [(0, 1), (1, 2)]).
                            MediaPipe의 PoseLandmark.value 인덱스를 사용.
        fps (float): 시각화 속도를 조절하기 위한 초당 프레임 수.
    """
    if not all_frames_3d_landmarks:
        print("시각화할 3D 랜드마크 데이터가 없습니다.")
        return

    # Open3D는 기본적으로 밀리미터 단위를 선호하지만, 여기서는 MediaPipe의 미터 단위를 그대로 사용합니다.
    # 필요하다면 여기서 * 1000.0 하여 밀리미터로 변환할 수 있습니다.
    # 예를 들어: initial_landmarks = all_frames_3d_landmarks[0] * 1000.0

    # 첫 프레임의 랜드마크로 초기 포인트 클라우드 및 라인셋 생성
    initial_landmarks = all_frames_3d_landmarks[0] # (num_markers, 3) 형태

    # 포인트 클라우드 객체 생성
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(initial_landmarks)
    # 모든 랜드마크를 빨간색으로 표시
    pcd.colors = o3d.utility.Vector3dVector(np.array([[1.0, 0.0, 0.0] for _ in range(initial_landmarks.shape[0])]))

    # 라인셋 객체 생성 (Open3D는 라인을 나타내기 위해 [point_idx1, point_idx2] 형태의 리스트를 원함)
    lines = []
    line_colors = [] # 각 연결선에 대한 색상 리스트
    for c_tuple in connections:
        lines.append([c_tuple[0], c_tuple[1]]) # 연결선의 점 인덱스 추가
        
        # 미리 정의된 CONNECTIONS_COLORS 딕셔너리에서 색상을 찾아 적용
        mapped_color = None
        for conn_key, color_rgb in CONNECTIONS_COLORS.items():
            # 양방향 연결을 모두 고려 (예: (11, 13) 또는 (13, 11))
            if (conn_key[0] == c_tuple[0] and conn_key[1] == c_tuple[1]) or \
               (conn_key[0] == c_tuple[1] and conn_key[1] == c_tuple[0]):
                mapped_color = [x / 255.0 for x in color_rgb] # RGB를 0-1 범위로 정규화
                break
        line_colors.append(mapped_color if mapped_color else [1.0, 1.0, 1.0]) # 기본값은 흰색

    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(initial_landmarks), # 초기 점 데이터
        lines=o3d.utility.Vector2iVector(lines) # 연결 정보
    )
    line_set.colors = o3d.utility.Vector3dVector(np.array(line_colors))

    # Open3D 시각화 도구 설정
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='3D Pose Animation', width=1024, height=768)
    
    # 초기 지오메트리 (포인트 클라우드와 라인셋)를 뷰어에 추가
    vis.add_geometry(pcd)
    vis.add_geometry(line_set)

    # 뷰 컨트롤 설정 (선택 사항: 초기 카메라 위치 및 줌 조정)
    ctr = vis.get_view_control()
    ctr.set_zoom(0.8) # 줌 레벨 조정
    # 초기 카메라 방향 설정 (예: 정면에서 바라보게)
    # ctr.set_front([0, 0, -1]) # Z 축 방향으로 앞을 바라봄
    # ctr.set_up([0, 1, 0])     # Y 축이 위를 향함
    # ctr.set_lookat([0, 0, 0]) # 원점을 중심으로 바라봄

    # 애니메이션 루프
    # 비디오의 FPS에 맞춰 프레임 간 지연 시간 설정
    delay_per_frame = 1.0 / fps if fps > 0 else 0.01 

    print("3D 시각화를 시작합니다. 창을 닫으면 종료됩니다.")
    for i, frame_landmarks_3d in enumerate(all_frames_3d_landmarks):
        # 포인트 클라우드와 라인셋의 점 데이터를 현재 프레임의 랜드마크로 업데이트
        pcd.points = o3d.utility.Vector3dVector(frame_landmarks_3d)
        line_set.points = o3d.utility.Vector3dVector(frame_landmarks_3d) # 라인셋도 점 데이터를 업데이트해야 함

        # 업데이트된 지오메트리를 뷰어에 반영
        vis.update_geometry(pcd)
        vis.update_geometry(line_set)
        
        # 사용자 인터랙션 (마우스 회전, 확대/축소 등) 처리
        vis.poll_events()
        # 렌더링 업데이트 (화면을 다시 그림)
        vis.update_renderer()

        # 프레임 속도에 맞춰 잠시 대기
        time.sleep(delay_per_frame)

    vis.destroy_window() # 모든 프레임을 표시한 후 시각화 창 닫기
    print("3D 시각화가 종료되었습니다.")

In [7]:
def draw_landmarks_custom(image, landmarks, image_width, image_height):
    """
    Draws custom-styled pose landmarks on the image.
    - Simplifies face to a single nose node.
    - Draws other body points as small gray dots.

    Args:
        image (np.array): The OpenCV BGR image frame to draw on.
        landmarks (mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList):
                    The pose landmarks detected by MediaPipe.
        image_width (int): Width of the image.
        image_height (int): Height of the image.
    """
    for idx, landmark in enumerate(landmarks.landmark):
        # Only draw if landmark visibility is good
        if landmark.visibility < min_draw_visibility:
            continue
        
        center_coordinates = (int(landmark.x * image_width), int(landmark.y * image_height))

        if idx == mp_pose.PoseLandmark.NOSE.value: # Nose: single face node
            cv2.circle(image, center_coordinates, 5, (255, 255, 255), -1) # White circle
        elif 1 <= idx <= 10: # Other facial landmarks (eyes, ears, mouth): don't draw
            pass
        else: # Body, arm, leg landmarks: small gray dot
            cv2.circle(image, center_coordinates, 2, (100, 100, 100), -1)

In [26]:
def draw_connections_custom(image, landmarks, image_width, image_height):
    """
    Draws custom color-coded pose connections (bones) on the image.

    Args:
        image (np.array): The OpenCV BGR image frame to draw on.
        landmarks (mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList):
                    The pose landmarks detected by MediaPipe.
        image_width (int): Width of the image.
        image_height (int): Height of the image.
    """
    for connection in mp_pose.POSE_CONNECTIONS:
        idx1, idx2 = connection
        
        if landmarks.landmark[idx1].visibility < min_draw_visibility or landmarks.landmark[idx2].visibility < min_draw_visibility:
            continue
        
        # Get color for the connection from the predefined map
        color = CONNECTIONS_COLORS.get(connection, None)
        if color is None: # Check if tuple order is reversed in map
            color = CONNECTIONS_COLORS.get((idx2, idx1), None)

        if color is not None:
            point1 = (int(landmarks.landmark[idx1].x * image_width), int(landmarks.landmark[idx1].y * image_height))
            point2 = (int(landmarks.landmark[idx2].x * image_width), int(landmarks.landmark[idx2].y * image_height))
            cv2.line(image, point1, point2, color, 2) # Line thickness 2

In [9]:
def draw_pose_on_frame(frame, pose_landmarks):
    """
    Orchestrates drawing pose landmarks and connections on a frame with custom styling.

    Args:
        frame (np.array): The OpenCV BGR image frame.
        pose_landmarks (mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList):
                        The pose landmarks detected by MediaPipe.

    Returns:
        np.array: The frame with the custom-drawn pose.
    """
    frame_with_pose = frame.copy()
    h, w, _ = frame.shape
    
    # Call functions to draw landmarks and connections
    draw_landmarks_custom(frame_with_pose, pose_landmarks, w, h)
    draw_connections_custom(frame_with_pose, pose_landmarks, w, h)
    
    return frame_with_pose

In [10]:
def export_to_c3d(output_filename, all_frames_3d_landmarks, fps, marker_names):
    if not all_frames_3d_landmarks:
        print("내보낼 3D 랜드마크 데이터가 없습니다.")
        return

    num_frames = len(all_frames_3d_landmarks)
    num_markers = all_frames_3d_landmarks[0].shape[0]

    points_data_all_frames = np.array(all_frames_3d_landmarks) * 1000.0 # meters to millimeters

    writer = c3d.Writer()

    for frame_idx in range(num_frames):
        current_frame_points_3d = points_data_all_frames[frame_idx]

        residuals_column = np.zeros((num_markers, 1), dtype=np.float32)

        points_with_residuals = np.hstack((current_frame_points_3d, residuals_column))
        points_with_residuals = np.hstack((points_with_residuals, residuals_column))

        writer.add_frames([(points_with_residuals, np.array([]))])

    with open(output_filename, 'wb') as handle:
        writer.write(handle)

    print(f"총 {num_frames} 프레임을 {output_filename}으로 성공적으로 내보냈습니다.")

In [11]:
def only_bone(frame, frame_with_pose):
    """
    Extracts only the drawn pose (bones and landmarks) from the frame.

    Args:
        frame (np.array): The original BGR image frame.
        frame_with_pose (np.array): The frame with pose, name, etc. drawn on it.

    Returns:
        np.array: A new frame containing only the drawn pose.
    """
    final_pose_only_frame = np.zeros_like(frame)
    # Compare pixels: if they're different, it means something was drawn.
    identical_pixels_mask = np.all(frame == frame_with_pose, axis=2)
    # Copy only the pixels that are different (where pose or name was drawn)
    final_pose_only_frame[~identical_pixels_mask] = frame_with_pose[~identical_pixels_mask]
    return final_pose_only_frame

## 3D 인터랙티브 시각화 클래스

In [12]:
class InteractivePoseVisualizer:
    def __init__(self, all_frames_3d_landmarks, connections, initial_fps):
        if not all_frames_3d_landmarks:
            raise ValueError("시각화할 3D 랜드마크 데이터가 없습니다.")

        self.all_frames_3d_landmarks = all_frames_3d_landmarks
        self.num_frames = len(all_frames_3d_landmarks)
        self.connections = connections

        self.current_frame_idx = 0
        self.is_playing = False
        self.playback_speed_factor = 1.0
        self.base_frame_delay = 1.0 / initial_fps if initial_fps > 0 else 0.033
        self.update_frame_delay()

        self.app = o3d.visualization.gui.Application.instance
        self.app.initialize()

        self.window = self.app.create_window("3D Interactive Pose Animation", 1024, 768)

        self.scene_widget = o3d.visualization.gui.SceneWidget()
        self.scene_widget.scene = o3d.visualization.rendering.Open3DScene(self.window.renderer)
        self.window.add_child(self.scene_widget)

        self.window.set_on_layout(self._on_layout)

        all_landmarks_flat = np.vstack(self.all_frames_3d_landmarks)
        self.scene_bbox = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(all_landmarks_flat))

        self.scene_widget.setup_camera(1.0, self.scene_bbox, [0, 0, -1])

        self.scene_widget.scene.set_background([0.1, 0.1, 0.1, 1.0])

        self.pcd_name = "landmarks_pcd"
        self.line_set_name = "connections_line_set"

        self.point_cloud_geometry = None
        self.line_set_geometry = None

        self._initialize_geometry()

        self.window.set_on_key(self._on_key_event)

        print("\n--- 3D 인터랙티브 포즈 애니메이션 조작 방법 ---")
        print("  Spacebar: 재생/일시정지")
        print("  'A' 또는 Left Arrow: 이전 프레임")
        print("  'D' 또는 Right Arrow: 다음 프레임")
        print("  'W' 또는 Up Arrow: 재생 속도 증가")
        print("  'S' 또는 Down Arrow: 재생 속도 감소")
        print("  'R': 카메라 뷰 초기화")
        print("  'Q': 시각화 종료")
        print("---------------------------------------------")

        self._update_geometry_for_frame(self.current_frame_idx)

    def _on_layout(self, layout_context):
        r = self.window.content_rect
        # SceneWidget의 frame 속성을 설정하면, 내부적으로 씬의 뷰포트와 크기를 자동으로 조절합니다.
        # 따라서 self.scene.set_view_size나 self.scene.set_viewport는 더 이상 필요하지 않습니다.
        self.scene_widget.frame = r
        # self.scene.set_view_size(r.width, r.height) # 제거
        # self.scene.set_viewport(r.x, r.y, r.width, r.height) # 제거

    def _initialize_geometry(self):
        initial_landmarks = self.all_frames_3d_landmarks[0]

        self.point_cloud_geometry = o3d.geometry.PointCloud()
        self.point_cloud_geometry.points = o3d.utility.Vector3dVector(initial_landmarks)
        self.point_cloud_geometry.colors = o3d.utility.Vector3dVector(
            np.array([[1.0, 0.0, 0.0] for _ in range(initial_landmarks.shape[0])])
        )

        lines = []
        line_colors = []
        for c_tuple in self.connections:
            lines.append([c_tuple[0], c_tuple[1]])
            mapped_color = None
            for conn_key, color_rgb in CONNECTIONS_COLORS.items():
                if (conn_key[0] == c_tuple[0] and conn_key[1] == c_tuple[1]) or \
                   (conn_key[0] == c_tuple[1] and conn_key[1] == c_tuple[0]):
                    mapped_color = [x / 255.0 for x in color_rgb]
                    break
            line_colors.append(mapped_color if mapped_color else [1.0, 1.0, 1.0])

        self.line_set_geometry = o3d.geometry.LineSet()
        self.line_set_geometry.points = o3d.utility.Vector3dVector(initial_landmarks)
        self.line_set_geometry.lines = o3d.utility.Vector2iVector(lines)
        self.line_set_geometry.colors = o3d.utility.Vector3dVector(np.array(line_colors))

        red_material = o3d.visualization.rendering.MaterialRecord()
        red_material.base_color = [1.0, 0.0, 0.0, 1.0]
        red_material.shader = "defaultLit"

        white_material = o3d.visualization.rendering.MaterialRecord()
        white_material.base_color = [1.0, 1.0, 1.0, 1.0]
        white_material.shader = "defaultLit"

        self.scene_widget.scene.add_geometry(self.pcd_name, self.point_cloud_geometry, red_material)
        self.scene_widget.scene.add_geometry(self.line_set_name, self.line_set_geometry, white_material)

    def _update_geometry_for_frame(self, frame_idx):
        if not (0 <= frame_idx < self.num_frames):
            print(f"경고: 프레임 인덱스 {frame_idx}가 범위를 벗어났습니다 (0-{self.num_frames-1}).")
            return

        frame_landmarks_3d = self.all_frames_3d_landmarks[frame_idx]

        self.point_cloud_geometry.points = o3d.utility.Vector3dVector(frame_landmarks_3d)
        # self.point_cloud_geometry.colors = o3d.utility.Vector3dVector(
        #     np.array([[1.0, 0.0, 0.0] for _ in range(frame_landmarks_3d.shape[0])])
        # )

        self.line_set_geometry.points = o3d.utility.Vector3dVector(frame_landmarks_3d)

    def update_frame_delay(self):
        self.current_frame_delay = self.base_frame_delay / max(0.01, self.playback_speed_factor)

    def _on_key_event(self, event):
        if event.type == o3d.visualization.gui.KeyEvent.Type.DOWN:
            key_code = event.key
            if key_code == o3d.visualization.gui.Key.SPACE:
                self._toggle_play_pause()
            elif key_code == o3d.visualization.gui.Key.A or key_code == o3d.visualization.gui.Key.LEFT:
                self._prev_frame()
            elif key_code == o3d.visualization.gui.Key.D or key_code == o3d.visualization.gui.Key.RIGHT:
                self._next_frame()
            elif key_code == o3d.visualization.gui.Key.W or key_code == o3d.visualization.gui.Key.UP:
                self._speed_up()
            elif key_code == o3d.visualization.gui.Key.S or key_code == o3d.visualization.gui.Key.DOWN:
                self._slow_down()
            elif key_code == o3d.visualization.gui.Key.R:
                self._reset_view()
            elif key_code == o3d.visualization.gui.Key.Q:
                self.window.close()
            return o3d.visualization.gui.Widget.EventCallbackResult.HANDLED
        return o3d.visualization.gui.Widget.EventCallbackResult.IGNORED

    def _toggle_play_pause(self):
        self.is_playing = not self.is_playing
        print(f"애니메이션: {'재생 중' if self.is_playing else '일시 정지'}")

    def _next_frame(self):
        self.is_playing = False # 수동 프레임 이동 시 재생 중지
        self.current_frame_idx = (self.current_frame_idx + 1) % self.num_frames
        self._update_geometry_for_frame(self.current_frame_idx)
        self.app.post_redraw() # 수동 프레임 이동 후 즉시 갱신 요청

    def _prev_frame(self):
        self.is_playing = False # 수동 프레임 이동 시 재생 중지
        self.current_frame_idx = (self.current_frame_idx - 1 + self.num_frames) % self.num_frames
        self._update_geometry_for_frame(self.current_frame_idx)
        self.app.post_redraw() # 수동 프레임 이동 후 즉시 갱신 요청

    def _speed_up(self):
        self.playback_speed_factor *= 1.2
        self.update_frame_delay()
        print(f"재생 속도: {self.playback_speed_factor:.2f}배")

    def _slow_down(self):
        self.playback_speed_factor /= 1.2
        if self.playback_speed_factor < 0.1: self.playback_speed_factor = 0.1
        self.update_frame_delay()
        print(f"재생 속도: {self.playback_speed_factor:.2f}배")

    def _reset_view(self):
        self.scene_widget.setup_camera(1.0, self.scene_bbox, [0, 0, -1])
        self.app.post_redraw() # 카메라 뷰 변경 후 즉시 갱신 요청
        print("카메라 뷰 초기화.")

    def run(self):
        # 이 함수는 post_to_main_thread에 의해 반복적으로 호출됩니다.
        def update_animation_loop():
            current_time = time.time()
            if self.is_playing and (current_time - self.last_frame_time) >= self.current_frame_delay:
                self.current_frame_idx = (self.current_frame_idx + 1) % self.num_frames
                self._update_geometry_for_frame(self.current_frame_idx)
                self.last_frame_time = current_time
                self.app.post_redraw() # 프레임 업데이트 후 다시 그리도록 요청

            # 애니메이션 루프를 계속 실행하기 위해 이 함수를 다시 스케줄링합니다.
            self.app.post_to_main_thread(self.window, update_animation_loop)

        self.last_frame_time = time.time()
        # 애니메이션 루프의 첫 호출을 스케줄링하여 시작합니다.
        self.app.post_to_main_thread(self.window, update_animation_loop)
        
        self.app.run()
        self.app.quit()
        print("3D 인터랙티브 시각화가 종료되었습니다.")

## 맞춤형 툴

### 관절 이름 맵

In [13]:
def get_joints(landmarks):
     return { "R_shoulder": landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
                "R_elbow": landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW],
                "R_wrist": landmarks[mp_pose.PoseLandmark.RIGHT_WRIST],
                "R_hip": landmarks[mp_pose.PoseLandmark.RIGHT_HIP],
                "R_knee": landmarks[mp_pose.PoseLandmark.RIGHT_KNEE],
                "R_ankle": landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE],
                "L_shoulder": landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER],
                "L_elbow": landmarks[mp_pose.PoseLandmark.LEFT_ELBOW],
                "L_wrist": landmarks[mp_pose.PoseLandmark.LEFT_WRIST],
                "L_hip": landmarks[mp_pose.PoseLandmark.LEFT_HIP],
                "L_knee": landmarks[mp_pose.PoseLandmark.LEFT_KNEE],
                "L_ankle": landmarks[mp_pose.PoseLandmark.LEFT_ANKLE],
                "R_pinky_tip": landmarks[mp_pose.PoseLandmark.RIGHT_INDEX],
                "L_pinky_tip": landmarks[mp_pose.PoseLandmark.LEFT_INDEX]}

### 각도 계산

In [14]:
def calculate_angle(vec1, vec2):
    # 코사인 값 계산
    # np.dot(ba, bc)는 내적, np.linalg.norm()은 벡터의 크기
    cosine_angle = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    # 아크코사인으로 라디안 각도 계산
    angle_radians = np.arccos(np.clip(cosine_angle, -1.0, 1.0)) # clip으로 부동 소수점 오차 방지

    # 라디안을 도로 변환
    angle_degrees = np.degrees(angle_radians)

    return angle_degrees

In [15]:
def calculate_angle_3(a, b, c):

    return calculate_angle(a - b, c - b)

In [16]:
def calculate_angle_4(a, b, c, d):
    v1 = np.array([a[0] - b[0], a[2] - b[2]]);
    v2 = np.array([c[0] - d[0], c[2] - d[2]])
    
    # 아크코사인으로 라디안 각도 계산
    angle_radians = np.arctan2(v1[0]*v2[1] - v1[1]*v2[0], v1[0]*v2[0] + v1[1]*v2[1])
    angle_degrees = np.degrees(angle_radians)

    return angle_degrees
    

### 투수용

In [17]:
def pitcher_tool(landmarks):

    joints = get_joints(landmarks)
    
    # --- 각도 계산 ---
    # 1. 오른쪽 팔꿈치 각도
    angle_R_elbow = calculate_angle_3(joints["R_shoulder"], joints["R_elbow"], joints["R_wrist"])
    angle_L_elbow = calculate_angle_3(joints["L_shoulder"], joints["L_elbow"], joints["L_wrist"])

    # 2. 오른쪽 어깨 각도 (몸통-어깨-팔꿈치"])
    angle_R_shoulder = calculate_angle_3(joints["L_shoulder"], joints["R_shoulder"], joints["R_elbow"])
    angle_L_shoulder = calculate_angle_3(joints["R_shoulder"], joints["L_shoulder"], joints["L_elbow"])

    # 3. 오른쪽 골반 각도 (몸통-골반-무릎"])
    angle_body_twist = calculate_angle_4(joints["R_shoulder"], joints["L_shoulder"], joints["R_hip"], joints["L_hip"])

    # 4. 오른쪽 무릎 각도
    angle_R_knee = calculate_angle_3(joints["R_hip"], joints["R_knee"], joints["R_ankle"])
    angle_L_knee = calculate_angle_3(joints["L_hip"], joints["L_knee"], joints["L_ankle"])

    return [ ["R Elbow", str(round(angle_R_elbow, 2))],
          ["L Elbow", str(round(angle_L_elbow, 2))],
          ["R Shoulder", str(round(angle_R_shoulder, 2))],
          ["L Shoulder", str(round(angle_L_shoulder, 2))],
          ["Body Twist", str(round(angle_body_twist, 2))],
          ["R Knee", str(round(angle_R_knee, 2))],
          ["L Knee", str(round(angle_L_knee, 2))]]  

### 타자용

In [18]:
def batter_tool(landmarks):
    joints = get_joints(landmarks)
    
    # --- 각도 계산 ---
    # 1. 팔꿈치 각도
    angle_R_elbow = calculate_angle_3(joints["R_shoulder"], joints["R_elbow"], joints["R_wrist"])
    angle_L_elbow = calculate_angle_3(joints["L_shoulder"], joints["L_elbow"], joints["L_wrist"])

    # 2. 어깨 각도
    angle_R_shoulder = calculate_angle_3(joints["L_shoulder"], joints["R_shoulder"], joints["R_elbow"])
    angle_L_shoulder = calculate_angle_3(joints["R_shoulder"], joints["L_shoulder"], joints["L_elbow"])

    # 3. 꼬임
    angle_body_twist = calculate_angle_4(joints["R_shoulder"], joints["L_shoulder"], joints["R_hip"], joints["L_hip"])

    # 4. 무릎 각도
    angle_R_knee = calculate_angle_3(joints["R_hip"], joints["R_knee"], joints["R_ankle"])
    angle_L_knee = calculate_angle_3(joints["L_hip"], joints["L_knee"], joints["L_ankle"])
    
    angle_R_wrist = calculate_angle_3(joints["R_elbow"], joints["R_wrist"], joints["R_pinky_tip"])
    angle_L_wrist = calculate_angle_3(joints["L_elbow"], joints["L_wrist"], joints["L_pinky_tip"])

    return [ ["R Elbow", str(round(angle_R_elbow, 2))],
          ["L Elbow", str(round(angle_L_elbow, 2))],
          ["R Shoulder", str(round(angle_R_shoulder, 2))],
          ["L Shoulder", str(round(angle_L_shoulder, 2))],
          ["Body Twist", str(round(angle_body_twist, 2))],
          ["R Knee", str(round(angle_R_knee, 2))],
          ["L Knee", str(round(angle_L_knee, 2))],
          ["R wrist", str(round(angle_R_wrist, 2))],
          ["L wrist", str(round(angle_L_wrist, 2))]]  

## 칼만 필터

In [19]:
# 칼만 필터 클래스 (새로 추가)
class LandmarkKalmanFilter:
    def __init__(self, num_landmarks=33, dim_state=6, dim_measurement=3, dt=1.0, process_noise_std=0.01, measurement_noise_std=0.1):
        # dim_state: 상태 벡터의 차원 (x, y, z, vx, vy, vz) = 6
        # dim_measurement: 측정 벡터의 차원 (x, y, z) = 3
        # dt: 시간 간격 (여기서는 프레임 간 간격, 1로 가정)
        
        self.filters = []
        for _ in range(num_landmarks):
            kf = KalmanFilter(dim_x=dim_state, dim_z=dim_measurement)

            # 상태 전이 행렬 (F): x = x + vx*dt, y = y + vy*dt, z = z + vz*dt
            kf.F = np.array([[1, 0, 0, dt, 0, 0],
                             [0, 1, 0, 0, dt, 0],
                             [0, 0, 1, 0, 0, dt],
                             [0, 0, 0, 1, 0, 0],
                             [0, 0, 0, 0, 1, 0],
                             [0, 0, 0, 0, 0, 1]])

            # 측정 행렬 (H): 측정값은 상태 벡터의 (x, y, z) 부분
            kf.H = np.array([[1, 0, 0, 0, 0, 0],
                             [0, 1, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0, 0]])

            # 공분산 행렬 (P): 초기 상태 불확실성 (클수록 초기 수렴 빠름)
            kf.P *= 1000.

            # 프로세스 노이즈 공분산 (Q): 모델 예측의 불확실성 (높을수록 필터가 측정값에 더 민감)
            # 랜드마크 움직임의 불확실성이 크다면 높게 설정 (예: 0.01)
            kf.Q = np.diag([process_noise_std**2]*3 + [process_noise_std**2]*3) # x,y,z 및 vx,vy,vz에 대한 노이즈

            # 측정 노이즈 공분산 (R): 측정값의 불확실성 (높을수록 필터가 측정값을 덜 신뢰)
            # MediaPipe 랜드마크의 노이즈가 심하면 높게 설정 (예: 0.1)
            kf.R = np.diag([measurement_noise_std**2]*3) # x,y,z 측정 노이즈

            # 초기 상태 벡터 (x): [x, y, z, vx, vy, vz]
            kf.x = np.zeros((dim_state, 1)) # 초기값은 첫 측정값으로 설정

            self.filters.append(kf)
        self.initialized = False # 필터 초기화 여부 플래그

    def initialize_state(self, first_landmarks_coords):
        # 첫 프레임에서 모든 랜드마크의 초기 상태를 설정 (측정값으로)
        for i, kf in enumerate(self.filters):
            if i < len(first_landmarks_coords): # 유효한 랜드마크 개수 범위 내에서
                # x, y, z는 첫 측정값으로, 속도(vx,vy,vz)는 0으로 초기화
                kf.x = np.array([[first_landmarks_coords[i][0]],
                                 [first_landmarks_coords[i][1]],
                                 [first_landmarks_coords[i][2]],
                                 [0.], [0.], [0.]])
        self.initialized = True

    def filter(self, current_landmarks_coords, visibility_scores=None, min_visibility_threshold=0.5):
        # current_landmarks_coords: 현재 프레임의 (num_landmarks, 3) numpy 배열
        # visibility_scores: 현재 프레임의 (num_landmarks,) numpy 배열 (0~1)
        
        filtered_coords = np.zeros_like(current_landmarks_coords)
        for i, kf in enumerate(self.filters):
            # predict 단계
            kf.predict()
            
            # update 단계: visibility 점수 기반으로 측정값 사용 여부 결정
            if visibility_scores is not None and visibility_scores[i] < min_visibility_threshold:
                # 랜드마크가 충분히 보이지 않으면 측정값을 사용하지 않고 예측값만 사용
                filtered_coords[i] = kf.x[:3].flatten() # 상태 벡터의 위치 부분만 사용
            else:
                # 랜드마크가 잘 보이면 측정값으로 업데이트
                kf.update(current_landmarks_coords[i].reshape(-1, 1))
                filtered_coords[i] = kf.x[:3].flatten()

        return filtered_coords

## 작동 함수

In [24]:
def pose_recognize_func(video_path, video_prename, tool):
    print("MediaPipe Pose를 초기화합니다...")
    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=model_complexity,
        enable_segmentation=False,
        min_detection_confidence=min_detection_confidence,
        min_tracking_confidence=min_tracking_confidence) as pose:

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"오류: 비디오 파일 {video_path}를 열 수 없습니다.")
            return

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        os.makedirs(output_dir, exist_ok=True)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        combined_out = cv2.VideoWriter(os.path.join(output_dir, video_prename + '_combined_output.mp4'), fourcc, fps, (frame_width, frame_height))
        bone_out = cv2.VideoWriter(os.path.join(output_dir, video_prename + '_bone_output.mp4'), fourcc, fps, (frame_width, frame_height))

        if not combined_out.isOpened() or not bone_out.isOpened():
            print("오류: 출력 비디오 파일을 생성할 수 없습니다. 코덱 또는 권한을 확인하세요.")
            cap.release()
            return

        all_frames_3d_landmarks = [] # 원본 (필터링 전) 랜드마크 저장용
        all_frames_filtered_3d_landmarks = [] # 필터링 후 랜드마크 저장용
        
        frame_count = 0
        progress_interval = max(1, total_frames // 100) # 진행률 표시를 위한 간격

        # 칼만 필터 초기화
        # MediaPipe Pose는 33개의 랜드마크를 가집니다.
        # process_noise_std와 measurement_noise_std 값은 튜닝이 필요할 수 있습니다.
        # 더 작은 값은 더 부드러운 결과를, 더 큰 값은 측정값에 더 빠르게 반응합니다.
        kalman_filter_processor = LandmarkKalmanFilter(
            num_landmarks=33, 
            process_noise_std=kalman_filter_process_noise_std,  # 프로세스 노이즈 표준 편차 (낮을수록 예측 신뢰)
            measurement_noise_std=kalman_filter_measurement_noise_std # 측정 노이즈 표준 편차 (낮을수록 측정 신뢰)
        )
        min_visibility_threshold = 0.6 # 랜드마크 가려짐 임계값

        print("비디오 처리 시작...")
        while cap.isOpened():
            ret, frame = cap.read() # 프레임 읽기
            if not ret:
                if total_frames > 0:
                    sys.stdout.write(f"\r처리 중: 100.00% ({total_frames}/{total_frames} 프레임)")
                    sys.stdout.flush()
                break

            frame_count += 1
            # 진행률 표시
            if total_frames > 0 and (frame_count == 1 or frame_count % progress_interval == 0 or frame_count == total_frames):
                progress_percent = (frame_count / total_frames) * 100
                sys.stdout.write(f"\r처리 중: {progress_percent:.2f}% ({frame_count}/{total_frames} 프레임)")
                sys.stdout.flush()

            # MediaPipe Pose 처리
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            results = pose.process(frame_rgb)
            frame_rgb.flags.writeable = True

            frame_with_pose = frame.copy()

            if results.pose_landmarks:
                # 월드 랜드마크 (3D 좌표) 추출
                current_raw_3d_landmarks_array = np.array([
                    [lmk.x, lmk.y, lmk.z]
                    for lmk in results.pose_world_landmarks.landmark
                ])
                all_frames_3d_landmarks.append(current_raw_3d_landmarks_array) # 원본 저장

                if apply_kalman_filter:
                    # 랜드마크 visibility (시야) 점수 추출
                    visibility_scores = np.array([lmk.visibility for lmk in results.pose_world_landmarks.landmark])
    
                    # 칼만 필터 초기화 또는 필터링
                    if not kalman_filter_processor.initialized:
                        kalman_filter_processor.initialize_state(current_raw_3d_landmarks_array)
                        filtered_landmarks_array = current_raw_3d_landmarks_array # 초기 프레임은 원본 그대로
                    else:
                        filtered_landmarks_array = kalman_filter_processor.filter(
                            current_raw_3d_landmarks_array, 
                            visibility_scores=visibility_scores, 
                            min_visibility_threshold=min_visibility_threshold
                        )
                else:
                    filtered_landmarks_array = current_raw_3d_landmarks_array
                        
                all_frames_filtered_3d_landmarks.append(filtered_landmarks_array) # 필터링된 랜드마크 저장

                # 필터링된 랜드마크를 사용하여 2D 포즈를 프레임에 그리기 (선택 사항: 원본으로 그릴 수도 있음)
                # 랜드마크를 그리기 위해 MediaPipe Drawing Spec에 다시 매핑해야 할 수 있습니다.
                # 여기서는 원본 랜드마크 결과를 사용하고, 계산만 필터링된 랜드마크로 합니다.
                frame_with_pose = draw_pose_on_frame(frame_with_pose, results.pose_landmarks)
                
                strs = tool(filtered_landmarks_array) 

                for i in range(len(strs)):
                    cv2.putText(frame_with_pose, strs[i][0] + ": " + strs[i][1], (10, frame.shape[0] - i * 15 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
                    
            # 결과 비디오 파일에 쓰기
            combined_out.write(frame_with_pose)
            final_pose_only_frame = only_bone(frame, frame_with_pose) # 이 함수도 필터링된 랜드마크로 그릴지 결정
            bone_out.write(final_pose_only_frame)

    print("\n비디오 객체를 해제합니다...")
    combined_out.release()
    bone_out.release()
    cap.release()

    # 필터링된 랜드마크와 FPS 반환
    return all_frames_filtered_3d_landmarks, fps

# 실행 부분

In [31]:
target_name = "이지헌1"
video_name = "raw_data/" + target_name + ".mov"
all_frames_3d_landmarks, fps = pose_recognize_func(video_name, target_name, batter_tool)

MediaPipe Pose를 초기화합니다...
비디오 처리 시작...
처리 중: 100.00% (661/661 프레임)
비디오 객체를 해제합니다...


In [24]:
# --- Export 3D data to C3D file ---
print("\nExporting 3D data to C3D file...")
c3d_output_path = os.path.join(output_dir, target_name + '_3d_pose.c3d')
export_to_c3d(c3d_output_path, all_frames_3d_landmarks, fps, ALL_POSE_MARKER_NAMES)
print("C3D export complete.")


Exporting 3D data to C3D file...
총 179 프레임을 result_data\이진우_3d_pose.c3d으로 성공적으로 내보냈습니다.
C3D export complete.


C:\Temp\Anaconda\Lib\site-packages\c3d\c3d.py:1210: UserWarning: missing parameter POINT:LABELS
  warnings.warn('missing parameter {}'.format(name))
C:\Temp\Anaconda\Lib\site-packages\c3d\c3d.py:1219: UserWarning: No analog data found in file.
  warnings.warn('No analog data found in file.')


In [ ]:
interactive_viz = InteractivePoseVisualizer(all_frames_3d_landmarks, mp_pose.POSE_CONNECTIONS, fps)
#interactive_viz.run()

In [ ]:
import open3d as o3d
import open3d.visualization.gui as gui
import open3d.visualization.rendering as rendering


app = gui.Application.instance
app.initialize()

window = app.create_window("Hello Open3D", 1024, 768)

# 씬 위젯 생성 및 추가
scene_widget = gui.SceneWidget()
scene_widget.scene = rendering.Open3DScene(window.renderer)
window.add_child(scene_widget)

# 카메라 설정
scene_widget.setup_camera(60, window.content_rect.width / window.content_rect.height,
                             o3d.geometry.AxisAlignedBoundingBox([-1, -1, -1], [1, 1, 1]))

    # 간단한 메시 추가
mesh = o3d.geometry.TriangleMesh.create_sphere()
mesh.compute_vertex_normals()
    
mat = rendering.MaterialRecord()
mat.shader = "defaultLit"
mat.base_color = [0.8, 0.8, 0.8, 1.0] # 회색

scene_widget.scene.add_geometry("sphere", mesh, mat)

# 레이아웃 설정
window.set_on_layout(lambda layout_context: scene_widget.set_frame(window.content_rect))

app.run()
app.quit()